#### *Purpose: check the Broadband Hyperspectral FalseColor image at certain caps, find optimal caps through manual implementation

# Load libraries and data

In [ ]:
#from gmodetector_py import XMatrix
from gmodetector_py import Hypercube
from gmodetector_py import WeightArray
from gmodetector_py import ImageChannel
from gmodetector_py import FalseColor
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob
from PIL import Image

In [ ]:
files = glob("./BROADBAND_FILE_DIRECTORY/*.hdr")

In [ ]:
file_path = "./BROADBAND_FILE_DIRECTORY/SPECIFIC_BROADBAND_FILE.hdr"

In [ ]:
test_cube = Hypercube(file_path,
                      min_desired_wavelength = 400,
                      max_desired_wavelength = 1000)

# Check spectra

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

mean_spectrum = np.mean(test_cube.hypercube, axis=(0, 1))


In [ ]:
# Convert wavelength strings to floats
wavelengths_float = np.array([float(w) for w in test_cube.wavelengths])

# Plot the mean spectrum
plt.plot(wavelengths_float, mean_spectrum)
plt.xlabel('Wavelength')
plt.ylabel('Mean Signal Intensity')
plt.title('Mean Spectrum')

# Set the x-axis tick marks
min_wavelength = int(np.min(wavelengths_float))
max_wavelength = int(np.max(wavelengths_float))
tick_marks = np.arange(min_wavelength - min_wavelength % 50, max_wavelength, 50)
plt.xticks(tick_marks)

plt.show()

# Make `FalseColor`

In [ ]:
stacked_component_image = FalseColor([ImageChannel(hypercube = test_cube,
                                                   desired_component_or_wavelength = "533.7419",
                                                   color = 'green',
                                                   cap = 563),
                                      ImageChannel(hypercube = test_cube,
                                                   desired_component_or_wavelength = "563.8288",
                                                   color = 'red',
                                                   cap = 904),
                                      ImageChannel(hypercube = test_cube, 
                                                   desired_component_or_wavelength = "500.0404",
                                                   color = 'blue',
                                                   cap = 406)])

#### Preprocess RGB & Hyperspecral FalseColor images

In [ ]:
# Load the RGB image and the hyperspectral image as FalseColor.image
hyperspectral_image = np.array(stacked_component_image.image)
rgb_image = cv2.imread('./FILE_DIRECTORY/SPECIFIC_RGB_FILE_rgb.jpg',
                       cv2.IMREAD_COLOR)

# Convert the images from OpenCV BGR format to RGB format
rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)

# Preprocess the images
rotated_rgb_image = np.rot90(rgb_image, k=3)  # Rotate RGB image by 270 degrees
rotated_hyperspectral_image = np.rot90(hyperspectral_image, k=2)  # Rotate hyperspectral image by 180 degrees
flipped_hyperspectral_image = np.fliplr(rotated_hyperspectral_image)  # Flip hyperspectral image horizontally

# Convert the preprocessed images to PIL Image objects
pil_rgb_image = Image.fromarray(rotated_rgb_image)
pil_hyperspectral_image = Image.fromarray(flipped_hyperspectral_image)

#### Display RGB and Hyperspectral FalseColor

In [ ]:
# Display broadband hyperspectral false color image with current caps shown above
pil_hyperspectral_image

In [ ]:
# Display RGB image loaded above
pil_rgb_image

# Optimized Caps Values: Manual Implementation

In [ ]:
import cv2
import numpy as np
from scipy.optimize import minimize, Bounds

def cost_function(caps, hyperspectral_image, rgb_image, test_cube):
    # Create a FalseColor image using the given cap parameters
    stacked_component_image = FalseColor([ImageChannel(hypercube=test_cube,
                                                       desired_component_or_wavelength="533.7419",
                                                       color='green',
                                                       cap=caps[0]),
                                          ImageChannel(hypercube=test_cube,
                                                       desired_component_or_wavelength="563.8288",
                                                       color='red',
                                                       cap=caps[1]),
                                          ImageChannel(hypercube=test_cube,
                                                       desired_component_or_wavelength="500.0404",
                                                       color='blue',
                                                       cap=caps[2])])

    # Convert the FalseColor image to a NumPy array
    stacked_np_image = np.array(stacked_component_image.image)

    # Resize the RGB image to match the dimensions of the hyperspectral image
    height, width, _ = stacked_np_image.shape
    resized_rgb_image = cv2.resize(rgb_image, (width, height), interpolation=cv2.INTER_AREA)

    # Compute the absolute difference between the resized RGB image and the FalseColor image
    diff = cv2.absdiff(stacked_np_image, resized_rgb_image)

    # Calculate the sum of the absolute differences as the cost
    cost = np.sum(diff)

    return cost

def nelder_mead(func, x0, args=(), bounds=None, min_step_size=5, maxiter=1000, tol=1e-4):
    x0 = np.asarray(x0)
    N = x0.size
    sim = np.zeros((N + 1, N), dtype=x0.dtype)
    sim[0] = x0
    for k in range(1, N + 1):
        y = np.array(x0, copy=True)
        y[k - 1] += min_step_size
        sim[k] = y

    allvecs = np.concatenate((sim, np.zeros((N + 1, 1))), axis=1)
    for k in range(N + 1):
        allvecs[k, N] = func(allvecs[k, :N], *args)

    ind = np.argsort(allvecs[:, N])
    allvecs = allvecs[ind]

    iterations = 0
    while iterations < maxiter:
        centroid = np.mean(allvecs[:-1, :N], axis=0)
        xr = centroid + 2.0 * (centroid - allvecs[-1, :N])
        fxr = func(xr, *args)

        if fxr < allvecs[0, N]:
            xe = centroid + 3.0 * (centroid - allvecs[-1, :N])
            fxe = func(xe, *args)

            if fxe < allvecs[0, N]:
                allvecs[-1, :N] = xe
                allvecs[-1, N] = fxe
            else:
                allvecs[-1, :N] = xr
                allvecs[-1, N] = fxr
        elif fxr < allvecs[-2, N]:
            allvecs[-1, :N] = xr
            allvecs[-1, N] = fxr

        else:
            if fxr < allvecs[-1, N]:
                allvecs[-1, :N] = xr
                allvecs[-1, N] = fxr

            xc = centroid + 0.5 * (allvecs[-1, :N] - centroid)
            fxc = func(xc, *args)

            if fxc <= allvecs[-1, N]:
                allvecs[-1, :N] = xc
                allvecs[-1, N] = fxc
            else:
                for k in range(1, N + 1):
                    allvecs[k, :N] = allvecs[0, :N] + (allvecs[k, :N] - allvecs[0, :N]) / 2
                    allvecs[k, N] = func(allvecs[k, :N], *args)

        ind = np.argsort(allvecs[:, N])
        allvecs = allvecs[ind]

        iterations += 1

        if np.std(allvecs[:, N]) <= tol:
            break

    return allvecs[0, :N]

# Initial cap values
initial_caps = [400, 600, 500]

# Set the bounds for your problem
bounds = Bounds([200, 200, 200], [900, 900, 900])

# Set the initial trust radius (step size)
trust_radius = 100

# Set the minimum step size
min_step_size = 5

# Use the custom Nelder-Mead algorithm to minimize the cost function
optimized_caps = nelder_mead(
    func=cost_function,
    x0=initial_caps,
    args=(hyperspectral_image, rgb_image, test_cube),
    bounds=bounds,
    min_step_size=min_step_size,
    maxiter=1000,
    tol=1e-4
)

print("Optimized cap values:", optimized_caps)
